Name: Sajjad Alizadeh <br>
#STD: 810197547

<div dir="rtl" style="color:black;font-size:18px;">
در همه حالات زیر ستون تیتر و ستون متن یکی در نظر گرفته شده است
<br>
    همچنین برای افزایش دقت از لگاریتم احتمال استفاده شده است.
    <br>
    تعریف prior:
    <br>
    احتمال اینکه یک کامنت پیشنهاد شده باشد یا نباشد بدون اینکه هیچ evidence ای موجود باشد. این احتمال برای حالت پیشنهاد شده برابر تعداد پیامهای پیشنهاد شده تقسیم بر کل پیامها و برای حالت پیشنهاد نشده برابر تعداد پیامهای پیشنهاد نشده تقسیم بر کل پیامها است
    <br>
    تعریف likelihood:
    <br>
    احتمال حضور یک پیام در یک کامنت به شرطی که پیشنهاد شده یا پیشنهاد نشده باشد. به عنوان مثال احتمال وجود کلمه «فدایی» در یک پیام به شرطی که آن پیام پیشنهاد شده باشد برابر تعداد تکرار این کلمه در کل پیامهای پیشنهاد شده تقسیم بر تعداد کل کلمات در کامنتهای پیشنهاد شده است.
    <br>
    تعریف evidence:
    حضور هر کلمه در یک کامنت یک evidence است.
    <br>
     تعریف posterior:
    <br>
    احتمال اینکه یک کامنت پیشنهاد شده باشد یا نباشد به شرط مجموعه ای از evidence ها. که همانطور که ذکر شد evidence همان حضور کلمات است. به عبارت دیگر احتمال اینکه یک کامنت پیشنهاد شده باشد به شرطی که کلمات آن را بدانیم (که میدانیم) و با استفاده از قانون بیز و شبکه های بیزی این احتمال را محاسبه میکنیم و بر اساس آن تصمیم گیری میکنیم
</div>

# No Pre Process, No Additive Smoothing

In [145]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import math

REC = 0
NOT_REC = 1
trained = [dict(), dict()]

def read_and_tokenize(path):
    file_content = pd.read_csv(path)
    file_content.replace('not_recommended', NOT_REC, inplace = True)
    file_content.replace('recommended', REC, inplace = True)
    train_data = file_content.values.tolist()
    for i in train_data:
        i[0] = word_tokenize(i[0])
        i[1] = word_tokenize(i[1])
        i[1] = i[1] + i[0]
    return train_data

train_data = read_and_tokenize("comment_train.csv")

size = [0, 0]
for i in train_data:
    rec_st = i[2]
    for j in i[1]:
        if j in trained[rec_st]: trained[rec_st][j] += 1
        else: trained[rec_st][j] = 1
        size[rec_st] += 1

for i in trained[REC]:
    trained[REC][i] = math.log(trained[REC][i] / size[REC])

for i in trained[NOT_REC]:
    trained[NOT_REC][i] = math.log(trained[NOT_REC][i] / size[NOT_REC])

rec_notrec_prob = [math.log(size[REC] / (size[REC] + size[NOT_REC])), math.log(size[NOT_REC] / (size[REC] + size[NOT_REC]))]

test_data = read_and_tokenize("comment_test.csv")

result = []
for comment in test_data:
    rec_prob = rec_notrec_prob[REC]
    not_rec_prob = rec_notrec_prob[NOT_REC]
    for word in comment[1]:
        if (word in trained[REC]) and (word in trained[NOT_REC]):
            rec_prob = rec_prob + trained[REC][word]
            not_rec_prob = not_rec_prob + trained[NOT_REC][word]
        elif (word in trained[REC]) and (word not in trained[NOT_REC]):
            rec_prob = rec_prob + trained[REC][word]
            not_rec_prob = float('-inf')
        elif (word not in trained[REC]) and (word in trained[NOT_REC]):
            rec_prob = float('-inf')
            not_rec_prob = not_rec_prob + trained[NOT_REC][word]
        elif (word not in trained[REC]) and (word not in trained[NOT_REC]):
             rec_prob = rec_prob + math.log(0.5)
             not_rec_prob = not_rec_prob + math.log(0.5)
    if rec_prob < not_rec_prob: result.append(NOT_REC)
    else: result.append(REC)
        
corrects = 0
correct_decected_rec = 0
all_detected_rec = 0
total_rec = 0
for i in range(len(test_data)):
    if test_data[i][2] == result[i]:
        corrects += 1
    if result[i] == REC:
        all_detected_rec += 1
        if test_data[i][2] == REC:
            correct_decected_rec += 1
    if test_data[i][2] == REC:
        total_rec += 1
precision = (correct_decected_rec / all_detected_rec) * 100
recall = (correct_decected_rec / total_rec) * 100
print("Accuracy: %f" % ((corrects / len(test_data)) * 100))
print("Precision: %f" % precision)
print("Recall: %f" % recall)
print("F1: %f" % ((2*precision*recall)/(precision+recall)))

Accuracy: 90.625000
Precision: 86.848073
Recall: 95.750000
F1: 91.082045


<div dir="rtl" style="color:black;font-size:18px;">
معیار precision معیار خوبی برای مقایسه نیست. فرض کنید هزار داده داریم که پانصدتای آنها recommanded هستند و الگوریتم دویست تا از پانصدتا را recommanded در نظر بگیرد. در این صورت معیار precision آن برابر صد خواهد شد در صورتی که واقعا پیش بینی خوبی صورت نگرفته است. به عبارتی فقط یک پنجم داده ها درست پیش بینی شده اند.
    <br>
    recall نیز معیار خوبی برای مقایسه نیست. همان مثال بالا که پانصد کامنت recommanded و پانصد کامنت not recommanded  را در نظر بگیرید. فرض کنید الگوریتم هر هزار کامنت را recommanded برچسب گذاری کند. در این صورت معیار recall برابر ۱۰۰ خواهد شد در صورتی که پیشبینی نیمی از داده ها اشتباه صورت گرفته است.
    <br>
     F1 برابر میانگین همساز دو حالت قبل است. همانطور که میدانیم این میانگین هنگامی کاربرد دارد که محاسبه میانگین نرخ ها اهمیت داشته باشد. در اصل بهترین میانگینی که میتوانیم داشته باشیم و میتواند ارائه خوبی از مدل ما باشد
</div>

# Pre Process, No Additive Smoothing

In [152]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import math
import csv

REC = 0
NOT_REC = 1
trained = [dict(), dict()]

def read_and_tokenize(path):
    file_content = pd.read_csv(path)
    file_content.replace('not_recommended', NOT_REC, inplace = True)
    file_content.replace('recommended', REC, inplace = True)
    train_data = file_content.values.tolist()
    for i in train_data:
        i[0] = word_tokenize(i[0])
        i[1] = word_tokenize(i[1])
        i[1] = i[1] + i[0]
    return train_data

train_data = read_and_tokenize("comment_train.csv")

# stemming and lemmatizing
stemmer = Stemmer()
lemmatizer = Lemmatizer()
for comment in train_data:
    for i in range(len(comment[1])):
        comment[1][i] = stemmer.stem(comment[1][i])
        comment[1][i] = lemmatizer.lemmatize(comment[1][i])

# read stopwords from file
stopwords = []
with open ('persian_stop_words.txt', 'r') as f:
    first_column = [row[0] for row in csv.reader(f,delimiter='\t')]
    stopwords += first_column[1:]

# remove stopwords
for comment in train_data:
    for i in range(len(comment[1]) - 1, -1, -1):
        if comment[1][i] in stopwords:
            comment[1].pop(i)
            

# count number of words and split recommand and not recommand
size = [0, 0]
for i in train_data:
    rec_st = i[2]
    for j in i[1]:
        if j in trained[rec_st]: trained[rec_st][j] += 1
        else: trained[rec_st][j] = 1
        size[rec_st] += 1

# use log(probality) to increase presicion.
for i in trained[REC]:
    trained[REC][i] = math.log(trained[REC][i] / size[REC])
for i in trained[NOT_REC]:
    trained[NOT_REC][i] = math.log(trained[NOT_REC][i] / size[NOT_REC])

rec_notrec_prob = [math.log(size[REC] / (size[REC] + size[NOT_REC])), math.log(size[NOT_REC] / (size[REC] + size[NOT_REC]))]

test_data = read_and_tokenize("comment_test.csv")

for comment in test_data:
    for i in range(len(comment[1])):
        comment[1][i] = stemmer.stem(comment[1][i])
        comment[1][i] = lemmatizer.lemmatize(comment[1][i])

for comment in test_data:
    for i in range(len(comment[1]) - 1, -1, -1):
        if comment[1][i] in stopwords: comment[1].pop(i)
        
result = []
for comment in test_data:
    rec_prob = rec_notrec_prob[REC]
    not_rec_prob = rec_notrec_prob[NOT_REC]
    for word in comment[1]:
        if (word in trained[REC]) and (word in trained[NOT_REC]):
            rec_prob = rec_prob + trained[REC][word]
            not_rec_prob = not_rec_prob + trained[NOT_REC][word]
        elif (word in trained[REC]) and (word not in trained[NOT_REC]):
            rec_prob = rec_prob + trained[REC][word]
            not_rec_prob = float('-inf')
        elif (word not in trained[REC]) and (word in trained[NOT_REC]):
            rec_prob = float('-inf')
            not_rec_prob = not_rec_prob + trained[NOT_REC][word]
        elif (word not in trained[REC]) and (word not in trained[NOT_REC]):
             rec_prob = rec_prob + math.log(0.5)
             not_rec_prob = not_rec_prob + math.log(0.5)
    if rec_prob < not_rec_prob: result.append(NOT_REC)
    else: result.append(REC)
        
corrects = 0
correct_decected_rec = 0
all_detected_rec = 0
total_rec = 0
for i in range(len(test_data)):
    if test_data[i][2] == result[i]:
        corrects += 1
    if result[i] == REC:
        all_detected_rec += 1
        if test_data[i][2] == REC:
            correct_decected_rec += 1
    if test_data[i][2] == REC:
        total_rec += 1
precision = (correct_decected_rec / all_detected_rec) * 100
recall = (correct_decected_rec / total_rec) * 100
print("Accuracy: %f" % ((corrects / len(test_data)) * 100))
print("Precision: %f" % precision)
print("Recall: %f" % recall)
print("F1: %f" % ((2*precision*recall)/(precision+recall)))

Accuracy: 91.125000
Precision: 87.643021
Recall: 95.750000
F1: 91.517324


<div dir="rtl" style="color:black;font-size:18px;">
در نتیجه بالا سه کار انجام داده ایم:
    <br>
اول)‌ stop words حذف شده است
    <br>
دوم) stemming صورت گرفته است.
    <br>
سوم) lemmatization صورت گرفته است.
</div>
<div dir="rtl" style="color:black;font-size:18px;">
مشاهده میشود نسبت به حالتی که هیچ پیش پردازشی انجام نشده است ۰.۵ درصد بهبود حاصل شده است
</div>
<div dir="rtl" style="color:black;font-size:18px;">
حذف stop words ها ضروری است زیرا زبان فارسی به گونه ای است که میتوان تعداد زیادی از آنها را بدون تغییر معنی به کاربرد و کلا این نوع کلمات تاثیری در تصمیم گیری ما نباید داشته باشد.
    <br>
اگر در کلمات stemming انجام ندهیم مقدار accuracy برابر ۹۰.۵ خواهد بود 
    <br>
اگر در کلمات lemmatization انجام ندهیم مقدار accuracy برابر ۹۰.۷۵ خواهد بود
    <br>
    اگر هیچ کدام از این دوکار را انجام ندهیم دقت به ۹۰.۳۷ خواهد رسید
    <br>
پس بهتر است هر دوی این کارها را انجام دهیم
<br>
    همچنین توجه کنید نیازی به نرمالایز کردن جملات نیست. دلیل اول آنکه بعضی کلمات مانند «جوش» به اشتباه به «جو» تعبیر میشوند و این اشکال از خود کتابخانه هضم است و دلیل دوم آنکه به عنوان مثال اگر نشانه های جمع بدون نیم فاصله نوشته شوند در قسمت حذف کردن استاپ وردها و اگر با نیم فاصله نوشته شوند در قسمت stemming حذف میشوند.
    <br>
    همچنین در این قسمت از استاپ وردهای خود هضم استفاده نکرده ایم. زیرا دقت پایین می آمد. از استاپ وردهایی که در اینترنت وجود داشت استفاده شده است.
</div>

# No Pre Process, Additive Smoothing

In [133]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import math

REC = 0
NOT_REC = 1
trained = [dict(), dict()]
size = [0, 0]

def read_and_tokenize(path):
    file_content = pd.read_csv(path)
    file_content.replace('not_recommended', NOT_REC, inplace = True)
    file_content.replace('recommended', REC, inplace = True)
    train_data = file_content.values.tolist()
    normalizer = Normalizer()
    for i in train_data:
        i[0] = word_tokenize(i[0])
        i[1] = word_tokenize(i[1])
        i[1] = i[1] + i[0]
    return train_data

train_data = read_and_tokenize("comment_train.csv")

for i in train_data:
    rec_st = i[2]
    for j in i[1]:
        if j in trained[rec_st]: trained[rec_st][j] += 1
        else: trained[rec_st][j] = 1
        size[rec_st] += 1

# additive smoothing
temp = []
for comment in trained[REC]:
    if comment not in trained[NOT_REC]:
        temp.append(comment)
if len(temp) != 0:
    for comment in trained[NOT_REC]:
        trained[NOT_REC][comment] += 1
    for t in temp:
        trained[NOT_REC][t] = 1
temp = []
for comment in trained[NOT_REC]:
    if comment not in trained[REC]:
        temp.append(comment)
if len(temp) != 0:
    for comment in trained[REC]:
        trained[REC][comment] += 1
    for t in temp:
        trained[REC][t] = 1
    

for i in trained[REC]:
    trained[REC][i] = math.log(trained[REC][i] / size[REC])

for i in trained[NOT_REC]:
    trained[NOT_REC][i] = math.log(trained[NOT_REC][i] / size[NOT_REC])

rec_notrec_prob = [math.log(size[REC] / (size[REC] + size[NOT_REC])), math.log(size[NOT_REC] / (size[REC] + size[NOT_REC]))]

test_data = read_and_tokenize("comment_test.csv")

result = []
for comment in test_data:
    rec_prob = rec_notrec_prob[REC]
    not_rec_prob = rec_notrec_prob[NOT_REC]
    for word in comment[1]:
        if (word in trained[REC]) and (word in trained[NOT_REC]):
            rec_prob = rec_prob + trained[REC][word]
            not_rec_prob = not_rec_prob + trained[NOT_REC][word]
        elif (word not in trained[REC]) and (word not in trained[NOT_REC]):
             rec_prob = rec_prob + math.log(0.5)
             not_rec_prob = not_rec_prob + math.log(0.5)
    if rec_prob < not_rec_prob: result.append(NOT_REC)
    else: result.append(REC)
        
corrects = 0
correct_decected_rec = 0
all_detected_rec = 0
total_rec = 0
for i in range(len(test_data)):
    if test_data[i][2] == result[i]:
        corrects += 1
    if result[i] == REC:
        all_detected_rec += 1
        if test_data[i][2] == REC:
            correct_decected_rec += 1
    if test_data[i][2] == REC:
        total_rec += 1
precision = (correct_decected_rec / all_detected_rec) * 100
recall = (correct_decected_rec / total_rec) * 100
print("Accuracy: %f" % ((corrects / len(test_data)) * 100))
print("Precision: %f" % precision)
print("Recall: %f" % recall)
print("F1: %f" % ((2*precision*recall)/(precision+recall)))

Accuracy: 93.750000
Precision: 92.892157
Recall: 94.750000
F1: 93.811881


<div dir="rtl" style="color:black;font-size:18px;">
در حالت بدون additive smoothing اگر کلمه ای جزو یکی از دسته های recommended و not recommended باشد و جزو دیگری نباشد 
تکلیف کامنت به صورت قطعی مشخص میشود. به عنوان مثال اگر کلمه دیجی کالا فقط در دسته recommended باشد و در دسته not recommended نباشد الگوریتم کل کامنت را به صورت recommended برچسب گذاری میکند
<br>
    در اصل این اتفاق وقتی می افتد که در الگوریتم وقتی کلمه ای وجود نداشت جای لگاریتم احتمال آن منفی بی نهایت گذاشتیم
    <br>
    برای حل این مشکل از additive smoothing استفاده میکنیم. به این صورت که کلماتی که فقط در یک گروه هستند را در یک لیست ذخیره میکنیم. سپس آنها را به گروهی که در آن نیستند با وزن یک اضافه میکنیم و به سایر کلمات نیز یک واحد اضافه میکنیم. این کار باعث میشود وقتی کلمه ای در یک گروه وجود ندارد باعث نشود کل کامنت به صورت قطعی عضو یک گروه شود. مشاهده میشود در این روش نسبت به حالت عادی ۳.۱ درصد افزایش دقت داریم.
<br>
    توجه کنید که عملیات ذکر شده additive smoothing به ازای آلفای یک است.
</div>

# Pre Process, Additive Smoothing

In [159]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import math

REC = 0
NOT_REC = 1
trained = [dict(), dict()]
size = [0, 0]

def read_and_tokenize(path):
    file_content = pd.read_csv(path)
    file_content.replace('not_recommended', NOT_REC, inplace = True)
    file_content.replace('recommended', REC, inplace = True)
    train_data = file_content.values.tolist()
    normalizer = Normalizer()
    for i in train_data:
        i[0] = word_tokenize(i[0])
        i[1] = word_tokenize(i[1])
        i[1] = i[1] + i[0]
    return train_data

train_data = read_and_tokenize("comment_train.csv")

# stemming and lemmatizing
stemmer = Stemmer()
lemmatizer = Lemmatizer()
for comment in train_data:
    for i in range(len(comment[1])):
        comment[1][i] = stemmer.stem(comment[1][i])
        #comment[1][i] = lemmatizer.lemmatize(comment[1][i])
        pass

# read stopwords from file
stopwords = []
with open ('persian_stop_words.txt', 'r') as f:
    first_column = [row[0] for row in csv.reader(f,delimiter='\t')]
    stopwords += first_column[1:]

# remove stopwords
for comment in train_data:
    for i in range(len(comment[1]) - 1, -1, -1):
        if comment[1][i] in stopwords:
            comment[1].pop(i)

for i in train_data:
    rec_st = i[2]
    for j in i[1]:
        if j in trained[rec_st]: trained[rec_st][j] += 1
        else: trained[rec_st][j] = 1
        size[rec_st] += 1

# additive smoothing
temp = []
for comment in trained[REC]:
    if comment not in trained[NOT_REC]:
        temp.append(comment)
if len(temp) != 0:
    for comment in trained[NOT_REC]:
        trained[NOT_REC][comment] += 1
    for t in temp:
        trained[NOT_REC][t] = 1
temp = []
for comment in trained[NOT_REC]:
    if comment not in trained[REC]:
        temp.append(comment)
if len(temp) != 0:
    for comment in trained[REC]:
        trained[REC][comment] += 1
    for t in temp:
        trained[REC][t] = 1
    

for i in trained[REC]:
    trained[REC][i] = math.log(trained[REC][i] / size[REC])

for i in trained[NOT_REC]:
    trained[NOT_REC][i] = math.log(trained[NOT_REC][i] / size[NOT_REC])

rec_notrec_prob = [math.log(size[REC] / (size[REC] + size[NOT_REC])), math.log(size[NOT_REC] / (size[REC] + size[NOT_REC]))]

test_data = read_and_tokenize("comment_test.csv")

# stemming and lemmatizing
for comment in test_data:
    for i in range(len(comment[1])):
        comment[1][i] = stemmer.stem(comment[1][i])
        #comment[1][i] = lemmatizer.lemmatize(comment[1][i])
    
# remove stopwords
for comment in test_data:
    for i in range(len(comment[1]) - 1, -1, -1):
        if comment[1][i] in stopwords: comment[1].pop(i)

result = []
for comment in test_data:
    rec_prob = rec_notrec_prob[REC]
    not_rec_prob = rec_notrec_prob[NOT_REC]
    for word in comment[1]:
        if (word in trained[REC]) and (word in trained[NOT_REC]):
            rec_prob = rec_prob + trained[REC][word]
            not_rec_prob = not_rec_prob + trained[NOT_REC][word]
        elif (word not in trained[REC]) and (word not in trained[NOT_REC]):
             rec_prob = rec_prob + math.log(0.5)
             not_rec_prob = not_rec_prob + math.log(0.5)
    if rec_prob < not_rec_prob: result.append(NOT_REC)
    else: result.append(REC)
        
corrects = 0
correct_decected_rec = 0
all_detected_rec = 0
total_rec = 0
c = 0
for i in range(len(test_data)):
    if test_data[i][2] == result[i]:
        corrects += 1
    if result[i] == REC:
        all_detected_rec += 1
        if test_data[i][2] == REC:
            correct_decected_rec += 1
    if test_data[i][2] == REC:
        total_rec += 1
    if c < 5 and test_data[i][2] != result[i]:
        print("BAD COMMENT")
        print(' '.join(map(str, test_data[i][1])) )
        c += 1
precision = (correct_decected_rec / all_detected_rec) * 100
recall = (correct_decected_rec / total_rec) * 100
print("Accuracy: %f" % ((corrects / len(test_data)) * 100))
print("Precision: %f" % precision)
print("Recall: %f" % recall)
print("F1: %f" % ((2*precision*recall)/(precision+recall)))

BAD COMMENT
خرید یه مد کار بکنه مشخص میشه کیف قطعات ور گود
BAD COMMENT
 رو مینویسه ول زیاد مناسب نیس رنگ میده یه وقتا موقع نوشتن زیاد مناسب نیس رنگ میده یه وقتا موقع نوشتن
BAD COMMENT
سلا راح کابل شارژ توصیه میشود ارز گوش شارژ وایرلس مجهز کنید نقد خرید
BAD COMMENT
جزو افراد سیزده ساله انواع فیل سرک اع روغن هوا اتاق استفاده میکرد ول تازگ متوجه اطلاع یاف فیل گاج باکیف  فیل سرک می‌باشد ه چنین قیم بمراتب مناسب ترب نسب سرک طرف فروشنده دا روغن فیلتر می‌فروخ واقعا اثب کرد گاج باکیف  سرک می‌باشد بررس فیل سرک
BAD COMMENT
سلا دوس استفاده چراغ چک تویوتا کمر ۲۰۰۷ خامو چراغ چک موتور خامو
Accuracy: 94.125000
Precision: 93.366093
Recall: 95.000000
F1: 94.175960


<div dir="rtl" style="color:black;font-size:18px;">
     توجه کنید در حالت بالا عملیات lemmatize را حذف کردیم زیرا باعث میشد دقت نسبت به حالتی pre process نداریم کاهش 
    پیدا کند 
<br>
    پنج کامنتی که به اشتباه تشخیص داده شده اند آورده شده است. (تایتل و متن باهم) یکی از دلایلی که ممکن است اشتباه تشخیص داده شود تایتل های نامربوط است. دلیل دیگر میتواند مشخص نبودن پیشنهاد دادن یا ندادن باشد. به عنوان مثال برنامه کامنت آخر را پیشنهاد نشده در نظر گرفته که در اصل پیشنهاد شده بوده است. تشخیص اینکه این کامنت پیشنهاد شده یا نشده برای انسان نیز دشوار است و عدم قطعیت دارد. دلیل دیگر میتواند stem کردن کلمات باشد که آنها را عوض میکند. مثلا سلام به سلا تبدیل شده است. دلیل دیگر میتواند ترکیب کلمات مثبت و منفی در یک کامنت باشد که سخت میشود تشخیص داد آن کامنت مثبت بوده یا منفی. دلیل دیگر ندانستن محصول مورد نظر است. مثلا در کامنت چهارم گفته میشود که گاج با کیفیت تر از سرکان است. اگر این کامنت برای گاج گذاشته شده باشد مثبت و اگر برای سرکان گذاشته شده باشد منفی است.
دلیل دیگر میتواند نزدیک بودن دو احتمال recommended بودن و not recommended بودن است که با توجه به داده های train شده ممکن است متفاوت شود. اگر این دو احتمال نزدیک به هم باشد تصمیم گیری سخت میشود.
</div>

<table>
  <tr>
    <th></th>
    <th>No Pre Process, No Additive Smoothing</th>
    <th>Pre Process, No Additive Smoothing</th>
    <th>No Pre Process, Additive Smoothing</th>
    <th>Pre Process, Additive Smoothing</th>
  </tr>
  <tr>
    <td>Accuracy</td>
    <td>90.625000</td>
    <td>91.125000</td>
    <td>93.750000</td>
    <td>94.125000</td>
  </tr>
  <tr>
    <td>Precision</td>
    <td>86.848073</td>
    <td>87.643021</td>
    <td>92.892157</td>
    <td>93.366093</td>
  </tr>
  <tr>
    <td>Recall</td>
    <td>95.750000</td>
    <td>95.750000</td>
    <td>94.750000</td>
    <td>95.000000</td>
  </tr>
  <tr>
    <td>F1</td>
    <td>91.082045</td>
    <td>91.517324</td>
    <td>93.811881</td>
    <td>94.175960</td>
  </tr>
</table>

<div dir="rtl" style="color:black;font-size:18px;">
    pre process آزاردهنده است. ممکن است تصور کنیم یکی از کارهایی که در راستای pre process میکنیم مفید است اما باید تست شود و هر pre process ای به بهتر شدن نتیجه کمک نمیکند.
    <br>
    additive smoothing همواره کمک میکند و باعث میشود دقت بیشتر شود.
    <br>
    اگر از هر دوی روشهای pre process و smoothing استفاده کنیم دقت از حالات دیگر بیشتر میشود. اما در این حالت نیز باید چک شود چه نوع pre process ای نتایج مارا بهتر میکند. به عنوان مثال در حالتی که smoothing نداشتیم استفاده از هر دوی stemming و lemmatize باعث بیشینه شدن دقت میشود اما در این حالت اگر از lemmatize استفاده نکنیم دقت بیشتر میشود و باید همه را تست کرد
</div>